NLP Model on New Data Rev 1.0

This program attempts to load a nlp classifier model and use it on a new/different data set other than that which created the model.

The export/import of the model and the vocabulary file are done through the pickle library
The extension of the model file and the vocabulary file is .sav

The first step is to load the data to be analyzed by the model. (.csv file)

The second step is to load the actual model and the vocabulary file. (.sav files) Both files are necessary.

Each are created when the model is created and saved through the use of pickle, which in essence saves the file as a binary file. Both of these files must be saved when the model is created.

https://medium.com/@maziarizadi/pickle-your-model-in-python-2bbe7dba2bbb
https://www.kdnuggets.com/2019/11/create-vocabulary-nlp-tasks-python.html

In [16]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk.classify
from nltk import NaiveBayesClassifier
analyzer = SentimentIntensityAnalyzer()
import time
import pickle

###########################################################
#### METHODS                                           ####
###########################################################

# displays a list of file with on a csv suffix       
def list_dir_files(relevant_path):
    # https://clay-atlas.com/us/blog/2019/10/27/python-english-tutorial-solved-unicodeescape-error-escape-syntaxerror/?doing_wp_cron=1618286551.1528689861297607421875
    #need to change \ to /

    import os
    
    included_extensions = ['csv']
    file_names = [fn for fn in os.listdir(relevant_path) # uses os.listdir to display only .csv files
              if any(fn.endswith(ext) for ext in included_extensions)]

    print('Path: ', relevant_path)

    for f in file_names:
        print(f)
        
# initializes the dataframe "df" and imports the csv into df; 
# the argument is the name/address of the file.
# https://stackoverflow.com/questions/33440805/pandas-dataframe-read-csv-on-bad-data
def getData(name):
    df1 = pd.DataFrame() # defines df1 as a dataframe
    df1 = pd.read_csv(name, header = 0)
    return df1


############################################################
####   MAIN                                             ####
############################################################

#Loads a specific model and vocabulary file:
#ModelFileToLoad = 'C:/Users/pstri/OneDrive/Documents/Personal/Kokoro/NLTK/Code Project/NLP Models/preprocessed r_a tech stockTwit 03112021Model.sav'
#VocabFileToLoad = 'C:/Users/pstri/OneDrive/Documents/Personal/Kokoro/NLTK/Code Project/NLP Models/preprocessed r_a tech stockTwit 03112021Vocab.sav'

ModelFileToLoad = 'preprocessed r_a tech stockTwit 03112021Model.sav'
VocabFileToLoad = 'preprocessed r_a tech stockTwit 03112021Vocab.sav'

loaded_model = pickle.load(open(ModelFileToLoad, 'rb'))
loaded_vocabulary = pickle.load(open(VocabFileToLoad, 'rb'))

'''#Calls getData to import the csv into the dataframe. This file is has the data that we want to perform the sentiment analysis on.
relevant_path = 'C:/Users/pstri/OneDrive/Documents/Personal/Kokoro/NLTK/Code Project/Scraped Files'

print('Here are the list of csv files to choose from: ')
list_dir_files(relevant_path) # method to list files in the directory where this file is found.
'''
time.sleep(1)

# name = input('\nWhat is the name of the csv file you want to import? \n')

name = 'preprocessed r_d r_a r_e_o tech stockTwit 03112021.csv'
dfAPI = getData(name)

fileName = name

####
# Need to convert sanitized data into vectors
####

# converts the df columns of body and the label (compound or sentiment_number) into one list for each column
#this is needed to be able to create the 
tech_twits_text = dfAPI['body'].tolist() #'body' is the column that contains the training data

''' There is no need for the following steps because they are captured in the 'loaded_vocabulary' file.
The vocabulary was created when the model was created.

from sklearn.feature_extraction.text import CountVectorizer
counter = CountVectorizer() #creates a CountVectorizer named counter
counter.fit(tech_twits_text) #teaches the counter the vocabulary

The loaded_vocabulary' file is set equal to counter and then counter is used to transform the data into a Count Vector.
This is just like when the model was created. The model then is applied to the Count Vector to determine sentiment.

'''
counter = loaded_vocabulary

tech_twits_text_counts = counter.transform(tech_twits_text) #transforms train_data to a Count Vector which will be the input to the model for sentiment analysis

predictions = loaded_model.predict(tech_twits_text_counts) #predicts the value (label/sentiment) from the tech_twits_text_counts 

print('predictions = ', predictions)


len(predictions) #gives the length of the array

type(predictions) #gives the data type of predictions

copy_predictions = predictions #makes a copy of predictions

df = pd.DataFrame(data = copy_predictions, columns = ['Sentiment']) #creates a dataframe from 'copy_predictions'

#display(df)

#itemizes the predicted sentiment values
bullish = 0
neutral = 0
bearish = 0

i = 0

while i < len(df):
    if df.iloc[i,0] == 1:
        bullish += 1
    elif df.iloc[i,0] == 0:
        neutral += 1
    elif df.iloc[i,0] == -1:
        bearish = 0
    
    i += 1
    
print('bullish = ', bullish)
print('neutral = ', neutral)
print('bearish = ', bearish)
    

predictions =  [1. 1. 1. ... 1. 0. 0.]
bullish =  663
neutral =  580
bearish =  0
